# Photovoltaics Interactive Tabletop
## Cristian Sorescu 879091

In [1]:
# imports

import numpy as np
import pandas as pd
import tsfresh
import matplotlib.pyplot as plt
from sklearn import preprocessing
from keras import optimizers, losses, activations, models
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from keras.layers import Dense, Input, Flatten, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, \
    concatenate
from sklearn.metrics import f1_score, accuracy_score

Using TensorFlow backend.


In [131]:
# functions

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = data
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

# cnn model
def get_model(_shape):
    nclass = 5
    inp = Input(shape=_shape)
    print(inp)
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid")(inp)
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = GlobalMaxPool1D()(img_1)
    img_1 = Dropout(rate=0.2)(img_1)

    dense_1 = Dense(64, activation=activations.relu, name="dense_1")(img_1)
    dense_1 = Dense(64, activation=activations.relu, name="dense_2")(dense_1)
    dense_1 = Dense(nclass, activation=activations.softmax, name="dense_3")(dense_1)

    model = models.Model(inputs=inp, outputs=dense_1)
    opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.sparse_categorical_crossentropy) # metrics=['accuracy']
    model.summary()
    return model

In [132]:
# gestures data
gestures = []

# fist formation gesture
handDetect = pd.read_csv('csv/hand_detected_1.csv', delim_whitespace=False, header=[0, 1], index_col=0)
gestures.append([handDetect, 'Hand Detetction'])
# fist formation gesture
fistForm = pd.read_csv('csv/fist_form_1.csv', delim_whitespace=False, header=[0, 1], index_col=0)
gestures.append([fistForm, 'Fist Formation'])
# clockwise motion gesture
clockwise = pd.read_csv('csv/clockwise_1.csv', delim_whitespace=False, header=[0, 1], index_col=0)
gestures.append([clockwise, 'Clockwise Motion'])
# counterclockwise motion gesture
counterclockwise = pd.read_csv('csv/counterclockwise_2.csv', delim_whitespace=False, header=[0, 1], index_col=0)
gestures.append([counterclockwise, 'Counterclockwise Motion'])
# continuous swipes motion gesture
swipe = pd.read_csv('csv/swipe_1.csv', delim_whitespace=False, header=[0, 1], index_col=0)
gestures.append([swipe, 'Singular Swipe'])
# singular swipe motion gesture
swipes = pd.read_csv('csv/swipes_1.csv', delim_whitespace=False, header=[0, 1], index_col=0)
gestures.append([swipes, 'Continuous Swipes'])
# raise and Lower hand motion
raiseLower = pd.read_csv('csv/raiseLower_1.csv', delim_whitespace=False, header=[0, 1], index_col=0)
gestures.append([raiseLower, 'Raise/Lower Hand Motion'])

# testing
gesture = fistForm

gesture = series_to_supervised(gesture, 79)
print(gesture.shape)

(14925, 80)


In [133]:
# rolling mean of gesture, reduce noise
rollingMean1 = gesture.rolling(window = 100).mean()
rollingMean1 = rollingMean1.dropna().drop_duplicates()

# define training data for gesture based on the rolling mean
y_train = np.array(abs(rollingMean1.values)).astype(np.double)[:,0]
x_train = np.array(abs(rollingMean1.values))[..., np.newaxis]

# y_train = np.expand_dims(y_train, axis=-1)
# print(y_train.shape)
# x_train = np.expand_dims(x_train, axis=0)

print(x_train.shape)
print(y_train.shape)

(14826, 80, 1)
(14826,)


In [134]:
# normalize the data, scale the y axis (valtage)
# min_max_scaler = preprocessing.MinMaxScaler()
# y_scaled = min_max_scaler.fit_transform(y_train)

# test plot
# plt.plot(x_train, y_train, label = "Foo")
# plt.plot(x_train, y_scaled, label = "Bar")
# plt.legend(loc='best')
# plt.show()

In [135]:
model = get_model(x_train[0, :].shape)

Tensor("input_15:0", shape=(None, 80, 1), dtype=float32)
Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        (None, 80, 1)             0         
_________________________________________________________________
conv1d_105 (Conv1D)          (None, 76, 16)            96        
_________________________________________________________________
conv1d_106 (Conv1D)          (None, 72, 16)            1296      
_________________________________________________________________
max_pooling1d_41 (MaxPooling (None, 36, 16)            0         
_________________________________________________________________
dropout_49 (Dropout)         (None, 36, 16)            0         
_________________________________________________________________
conv1d_107 (Conv1D)          (None, 34, 32)            1568      
_________________________________________________________________
co

In [136]:
# early = EarlyStopping(monitor="accuracy", mode="max", patience=5, verbose=1)
# redonplat = ReduceLROnPlateau(monitor="accuracy", mode="max", patience=3, verbose=2)
# callbacks_list = [early, redonplat]

# fit data into model, training
model.fit(x_train, y_train, epochs=30, verbose=1)

Epoch 1/30
14826/14826 [==============================] - 10s 680us/step - loss: 0.0240
Epoch 2/30
14826/14826 [==============================] - 9s 604us/step - loss: 0.0000e+00
Epoch 3/30
14826/14826 [==============================] - 9s 603us/step - loss: 0.0000e+00
Epoch 4/30
14826/14826 [==============================] - 9s 610us/step - loss: 0.0000e+00
Epoch 5/30
14826/14826 [==============================] - 9s 604us/step - loss: 0.0000e+00
Epoch 6/30
14826/14826 [==============================] - 9s 611us/step - loss: 0.0000e+00
Epoch 7/30
14826/14826 [==============================] - 9s 610us/step - loss: 0.0000e+00
Epoch 8/30
14826/14826 [==============================] - 9s 611us/step - loss: 0.0000e+00
Epoch 9/30
14826/14826 [==============================] - 9s 607us/step - loss: 0.0000e+00
Epoch 10/30
14826/14826 [==============================] - 9s 611us/step - loss: 0.0000e+00
Epoch 11/30
14826/14826 [==============================] - 9s 613us/step - loss: 0.0000e+00


In [148]:
gesture_test = pd.read_csv('csv/fist_form_2.csv', delim_whitespace=False, header=[0, 1], index_col=0)
gesture_test = series_to_supervised(gesture_test, 79)
# print(gesture_test.shape)

# rolling mean of gesture, reduce noise
rollingMean_test = gesture_test.rolling(window = 100).mean()
rollingMean_test = rollingMean_test.dropna().drop_duplicates()

# define training data for gesture based on the rolling mean
y_test = np.array(abs(rollingMean_test.values)).astype(np.double)[:,0]
x_test = np.array(abs(rollingMean_test.values))[..., np.newaxis]

# print(x_test.shape)
# print(y_test.shape)

pred_test = model.predict(x_test)
pred_test = np.argmax(pred_test, axis=-1)

f1 = f1_score(np.around(y_test), pred_test, average="macro")

print("Test f1 score : %s "% f1)

acc = accuracy_score(np.around(y_test), pred_test)

print("Test accuracy score : %s"% acc)

Test f1 score : 1.0 
Test accuracy score : 1.0


In [128]:
# tests

# inp =  Input(shape=(5, 1))
# conv = Convolution1D(filters=2, kernel_size=2)(inp)
# pool = MaxPool1D(pool_size=2)(conv)
# flat = Flatten()(pool)
# dense = Dense(1)(flat)
# model = models.Model(inp, dense)
# model.compile(loss='mse', optimizer='adam')

# #print(model.summary())

# # get some data
# X = np.expand_dims(np.random.randn(10, 5), axis=2)
# y = np.random.randn(10, 1)

# print(X.shape)
# print(y.shape)

# # fit model
# model.fit(X, y)

# Plot Gestures

In [ ]:
plt.rcParams.update({'font.size': 32})

for gesture in gestures:
    gestureDF, title = gesture
    #gestureDF.columns = gestureDF.columns.droplevel(1)
    plotTitle = title

    rollingMean = gestureDF.rolling(window = 100).mean()
    rollingMean = rollingMean.dropna().drop_duplicates()
    rollingStd = gestureDF.rolling(window = 12).std()
    #plt.plot(gestureDF, color = 'blue', label = 'Original')
    plt.plot(rollingMean, color = 'red', label = 'Rolling Mean')
    #plt.plot(rollingStd, color = 'black', label = 'Rolling STD')
    #plt.legend(loc = 'best', prop={'size': 13})
    #plt.title(plotTitle)
    plt.xlabel('Time (s)')
    plt.ylabel('Voltage (V)')
    #minR = round(gestureDF.min().to_numpy()[0], 1)
    #maxR = round(gestureDF.max().to_numpy()[0], 1)
    #plt.yticks(np.arange(minR, maxR, step=0.3)) 
    plt.show()

In [ ]:
# plot the data

#fistForm.shape
#fistForm.head(3)
gestures[:5]

#handDetect['Channel A'].plot(linewidth=1);
#fistForm['Channel A'].plot(linewidth=1);
#clockwise['Channel A'].plot(linewidth=1);

# tsfresh Tests

In [ ]:
# extract features
fistForm.reset_index(inplace=True)
fistForm_features = tsfresh.extract_features(fistForm, column_id='index', n_jobs=4)

#fistForm_features = tsfresh.extract_features(fistForm)

#clockwise_features = tsfresh.extract_features(clockwise)

#counterclockwise_features = tsfresh.extract_features(counterclockwise)

#swipes_features = tsfresh.extract_features(swipes)

#swipe_features = tsfresh.extract_features(swipe)

#raiseLower_features = tsfresh.extract_features(raiseLower)

In [ ]:
from tsfresh.utilities.dataframe_functions import impute

impute(handDetect_features)

In [ ]:
#handDetect_features.head()
#handDetect.resample('3T').sum()
#handDetect.describe()

In [19]:
for gesture in gestures:
    gestureDF, title = gesture
    print(round(gestureDF.max().to_numpy()[0], 1))

0.1
-0.1
-1.1
0.1
0.1
-0.9
-0.2


In [ ]:
# tests

# import mass_ts as mts

# ts = handDetect3.to_numpy()
# q = handDetect.to_numpy()

# print(handDetect3[:,0])

# # mass
# distances = mts.mass(ts[:,0], q[:,0])

# # mass2
# distances = mts.mass2(ts[:,0], q[:,0])

# # mass3
# #distances = mts.mass3(ts[:,0], q[:,0], 256)

# # mass2_batch
# # start a multi-threaded batch job with all cpu cores and give me the top 5 matches.
# # note that batch_size partitions your time series into a subsequence similarity search.
# # even for large time series in single threaded mode, this is much more memory efficient than
# # MASS2 on its own.
# batch_size = 10000
# top_matches = 5
# n_jobs = 3
# indices, distances = mts.mass2_batch(ts[:,0], q[:,0], batch_size, 
#     top_matches=top_matches, n_jobs=n_jobs)

# # find minimum distance
# min_idx = np.argmin(distances)

# # find top 4 motif starting indices
# k = 4
# exclusion_zone = 25
# top_motifs = mts.top_k_motifs(distances, k, exclusion_zone)

# # find top 4 discord starting indices
# k = 4
# exclusion_zone = 25
# top_discords = mts.top_k_discords(distances, k, exclusion_zone)

# Wavelet analysis

In [3]:
import pywt
x = handDetect.values
# Discrete Wavelet Transform
#cA, cD = pywt.dwt(x, 'db2')
#x2 = pywt.idwt(cA, cD, 'db2')
x

array([[0.00000000e+00, 1.14749600e-01],
       [3.33340000e-04, 9.76592300e-02],
       [6.66690000e-04, 1.01016300e-01],
       ...,
       [5.00049351e+00, 6.40888700e-02],
       [5.00082685e+00, 4.36414700e-02],
       [5.00116020e+00, 3.32651700e-02]])

In [16]:
# for mode_name in ['zero', 'constant', 'symmetric', 'reflect', 'periodic', 'smooth', 'periodization']:
#     mode = getattr(pywt.Modes, mode_name)
#     cA, cD = pywt.dwt(x, 'db2', mode)
#     print("Mode: %d (%s)" % (mode, mode_name))

Mode: 0 (zero)
Mode: 2 (constant)
Mode: 1 (symmetric)
Mode: 6 (reflect)
Mode: 4 (periodic)
Mode: 3 (smooth)
Mode: 5 (periodization)


In [4]:
wp = pywt.WaveletPacket2D(data=x, wavelet='db1')

In [5]:
wp.data

array([[0.00000000e+00, 1.14749600e-01],
       [3.33340000e-04, 9.76592300e-02],
       [6.66690000e-04, 1.01016300e-01],
       ...,
       [5.00049351e+00, 6.40888700e-02],
       [5.00082685e+00, 4.36414700e-02],
       [5.00116020e+00, 3.32651700e-02]])

In [6]:
wp['a'].data

array([[0.10637109],
       [0.11298891],
       [0.1331874 ],
       ...,
       [5.09976086],
       [5.06700978],
       [5.03944685]])